In [4]:
import paddle
import paddle.nn.functional as F
import numpy as np
import pandas as pd
import os
from paddle import nn
from PIL import Image
from paddle.distributed import fleet, get_rank
from visualdl import LogWriter

IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCH_NUM = 5
logwriter = LogWriter(logdir='./runs')

print(paddle.__version__)

2.4.0


In [12]:
# 定义数据集
class MyDataset(paddle.io.Dataset):
    def __init__(self, img_dir='data/PALM-Training400/', csv_dir='data/Classification.csv') -> None:
        super(MyDataset, self).__init__()
        if csv_dir is None:
            self.csvfile = None
            self.filedir = os.listdir(img_dir)
        else:
            self.csvfile = pd.read_csv(csv_dir)
        self.imgpath = img_dir
        pass
    def __len__(self):
        if self.csvfile is None:
            return len(self.filedir)
        else:
            return len(self.csvfile)
        pass
    def __getitem__(self, idx):
        if self.csvfile is None:
            img = np.reshape((np.array(Image.open(self.imgpath+os.sep+self.filedir[idx]).resize((IMAGE_SIZE,IMAGE_SIZE))).astype('float32')),(3,IMAGE_SIZE,IMAGE_SIZE))/256.
            lab = self.filedir[idx]
        else:
            img = np.reshape((np.array(Image.open(self.imgpath+os.sep+self.csvfile['imgName'][idx]).resize((IMAGE_SIZE,IMAGE_SIZE))).astype('float32')),(3,IMAGE_SIZE,IMAGE_SIZE))/256.
            lab = np.array(self.csvfile['Label'][idx]).astype('float32')
        return img,lab
    pass
mydataset = MyDataset('Train/fundus_image/', 'Train/Classification.csv')

In [6]:
# 定义网络结构
def vgg_block(num_convs, in_channels, out_channels):
    net = [nn.Conv2D(in_channels=in_channels,out_channels=out_channels,kernel_size=3,padding=1),nn.ReLU()]
    for i in range(num_convs-1):
        net.append(nn.Conv2D(out_channels=out_channels,in_channels=out_channels,kernel_size=3,stride=1,padding=1))
        net.append(nn.ReLU())
    net.append(nn.MaxPool2D(kernel_size=2))
    return nn.Sequential(*net)

def vgg_stack(num_convs,channels):
    net = []
    for n,c in zip(num_convs,channels):
        in_c = c[0]
        out_c = c[1]
        net.append(vgg_block(n,in_c,out_c))
    return nn.Sequential(*net)

class VGG(paddle.nn.Layer):
    def __init__(self,vgg_net) -> None:
        super(VGG,self).__init__()
        self.conv = vgg_stack(vgg_net[0],vgg_net[1])
        self.line = nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,1)
        )
        
    def forward(self,x):
        x = self.conv(x)
        x = paddle.flatten(x, 1, -1)
        x = self.line(x)
        return x

net=VGG([[2,2,3,3,3], [[3,64],[64,128],[128,256],[256,512],[512,512]]])

W0203 15:29:47.017560  6991 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0203 15:29:47.022044  6991 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


In [4]:
def train_pm(model, optimizer):
    # 开启0号GPU训练
    paddle.device.set_device('gpu:0')

    print('start training ... ')
    model.train()
    # 定义数据读取器，训练数据读取器和验证数据读取器
    train_loader = paddle.io.DataLoader(mydataset,batch_size=BATCH_SIZE,shuffle=True,drop_last=True)
    valid_loader = paddle.io.DataLoader(mydataset,batch_size=BATCH_SIZE,shuffle=True,drop_last=True)
    for epoch in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            x_data, y_data = data
            img = paddle.to_tensor(x_data)
            label = paddle.reshape(paddle.to_tensor(y_data),(-1,1))
            # 运行模型前向计算，得到预测值
            logits = model(img)
            loss = F.binary_cross_entropy_with_logits(logits, label)
            avg_loss = paddle.mean(loss)

            if batch_id % 5 == 4:
                print("epoch: {}, batch_id: {}, loss is: {:.4f}".format(epoch, batch_id, float(avg_loss.numpy())))
            # 反向传播，更新权重，清除梯度
            avg_loss.backward()
            optimizer.step()
            optimizer.clear_grad()

        model.eval()
        accuracies = []
        losses = []
        for batch_id, data in enumerate(valid_loader()):
            x_data, y_data = data
            img = paddle.to_tensor(x_data)
            label = paddle.reshape(paddle.to_tensor(y_data),(-1,1))
            # 运行模型前向计算，得到预测值
            logits = model(img)
            # 二分类，sigmoid计算后的结果以0.5为阈值分两个类别
            # 计算sigmoid后的预测概率，进行loss计算
            pred = F.sigmoid(logits)
            loss = F.binary_cross_entropy_with_logits(logits, label)
            # 计算预测概率小于0.5的类别
            pred2 = pred * (-1.0) + 1.0
            # 得到两个类别的预测概率，并沿第一个维度级联
            pred = paddle.concat([pred2, pred], axis=1)
            acc = paddle.metric.accuracy(pred, paddle.cast(label, dtype='int64'))

            accuracies.append(acc.numpy())
            losses.append(loss.numpy())
        print("[validation] accuracy/loss: {:.4f}/{:.4f}".format(np.mean(accuracies), np.mean(losses)))
        model.train()

        paddle.save(model.state_dict(), 'palmp.pdparams')
        paddle.save(optimizer.state_dict(), 'palmp.pdopt')

In [5]:
# 创建模型
model =VGG([[2,2,3,3,3], [[3,64],[64,128],[128,256],[256,512],[512,512]]])
# opt = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
opt = paddle.optimizer.Momentum(learning_rate=0.001, momentum=0.9, parameters=model.parameters())

# 启动训练过程
train_pm(model, opt)

start training ... 
epoch: 0, batch_id: 0, loss is: 1.1686
epoch: 0, batch_id: 5, loss is: 0.6386
epoch: 0, batch_id: 10, loss is: 0.7067
epoch: 0, batch_id: 15, loss is: 0.6174
epoch: 0, batch_id: 20, loss is: 0.5037
[validation] accuracy/loss: 0.7312/0.4496
epoch: 1, batch_id: 0, loss is: 0.4178
epoch: 1, batch_id: 5, loss is: 0.1879
epoch: 1, batch_id: 10, loss is: 0.3120
epoch: 1, batch_id: 15, loss is: 0.2449
epoch: 1, batch_id: 20, loss is: 0.3972
[validation] accuracy/loss: 0.9300/0.2949
epoch: 2, batch_id: 0, loss is: 0.3461
epoch: 2, batch_id: 5, loss is: 0.2982
epoch: 2, batch_id: 10, loss is: 0.2426
epoch: 2, batch_id: 15, loss is: 0.1439
epoch: 2, batch_id: 20, loss is: 0.1721
[validation] accuracy/loss: 0.9300/0.1992
epoch: 3, batch_id: 0, loss is: 0.2212
epoch: 3, batch_id: 5, loss is: 0.1581
epoch: 3, batch_id: 10, loss is: 0.1064
epoch: 3, batch_id: 15, loss is: 0.1153
epoch: 3, batch_id: 20, loss is: 0.2616
[validation] accuracy/loss: 0.9488/0.1437
epoch: 4, batch_id: 

KeyboardInterrupt: 

In [39]:
def predict(model, dataloader):
    model.eval()
    pred_list = np.array([])
    file_list = np.array([])
    for idx, data in enumerate(dataloader):
        print('\r{}/{}'.format(1+idx,len(dataloader)),end='')
        x_data, filename = data
        img = paddle.to_tensor(x_data)
        # 运行模型前向计算，得到预测值
        logits = model(img)
        # 二分类，sigmoid计算后的结果以0.5为阈值分两个类别
        # 计算sigmoid后的预测概率，进行loss计算
        pred = F.sigmoid(logits)
        pred_list = np.append(pred_list, pred.numpy().ravel())
        file_list = np.append(file_list, filename)
    return pred_list, file_list

In [40]:
# 读取模型
model =VGG([[2,2,3,3,3], [[3,64],[64,128],[128,256],[256,512],[512,512]]])
model.set_state_dict(paddle.load('model/palmp.pdparams'))
testdataset = MyDataset('PALM-Testing400-Images',None)
testdataloader = paddle.io.DataLoader(testdataset,shuffle=False,drop_last=False,batch_size=BATCH_SIZE)

pred_list,file_list = predict(model, testdataloader)
pd.DataFrame(np.c_[file_list,pred_list],columns=['FileName','PM Risk']).to_csv('Classification_Results.csv',index=False)

13/13